<a href="https://colab.research.google.com/github/nezhalahnech/TFM/blob/main/my_implementation_TFM_nezha_lahnech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Libraries***

In [ ]:
!pip install pycryptodome

In [ ]:
!pip install base58

In [ ]:
!pip install utils

In [ ]:
import hashlib
from datetime import datetime
import json
import time
import math
import base58
from Crypto.PublicKey import RSA
from Crypto.Signature import pkcs1_15
from Crypto.Hash import SHA256, RIPEMD160
import binascii


# ***Validation of Blockchain and Transaction***

In [ ]:

def calculate_ext_hash(transaccion, prev_hash_str, timestamp):
    transaccion_str = json.dumps(transaccion, sort_keys=True, default=str)
    data = transaccion_str + prev_hash_str + str(timestamp)
    hash_object = hashlib.sha256(data.encode())
    return hash_object.hexdigest()

class Bloque:
    def __init__(self, transaccion, timestamp, bloque_previo=None):
        self.transaccion = transaccion
        self.timestamp = timestamp
        self.bloque_previo = bloque_previo
        self.cryptographic_hash = self.calculate_hash()

    def calculate_hash(self):
        prev_hash_str = str(self.bloque_previo.cryptographic_hash) if self.bloque_previo else ""
        return calculate_ext_hash(self.transaccion, prev_hash_str, self.timestamp)

def is_valid_transaction(transaccion, bloque):
    required_keys = ['From', 'To', 'Amount']
    if not all(key in transaccion for key in required_keys):
        return False
    sender_balance = get_account_balance(transaccion['From'], bloque.bloque_previo)
    if sender_balance < transaccion['Amount']:
        return False
    return True

def is_valid_blockchain(cadena):
    bloque = cadena
    while bloque.bloque_previo:
        transaccion = bloque.transaccion
        if not is_valid_transaction(transaccion, bloque):
            return False
        if  (bloque.cryptographic_hash !=  calculate_ext_hash (bloque.transaccion,str(bloque.bloque_previo.cryptographic_hash),bloque.timestamp)):
            return False
        bloque = bloque.bloque_previo
    if (bloque.transaccion['From'] == 'genesis') and (bloque.transaccion['Amount'] >=0) and (bloque.cryptographic_hash ==  calculate_ext_hash (bloque.transaccion,"",bloque.timestamp)):
       return True
    else:
       return False

def get_account_balance(account, bloque):
    balance = 0
    while bloque:
        transaccion = bloque.transaccion
        sender = transaccion['From']
        recipient = transaccion['To']
        amount = transaccion['Amount']
        if sender == account:
            balance -= amount
        if recipient == account:
            balance += amount
        bloque = bloque.bloque_previo
    return balance

# Create the initial transaction
initial_transaction = {
    'From': 'genesis',
    'To': 'account1',
    'Amount': 1000
}

# Create the initial block
bloque_inicial = Bloque(initial_transaction, datetime.now().timestamp())

# Create subsequent blocks with transactions and timestamps
transaction1= {
    'From': 'account1',
    'To': 'account2',
    'Amount': 100
}
bloque1 = Bloque(transaction1, datetime.now().timestamp(), bloque_inicial)

transaction2 = {
    'From': 'account2',
    'To': 'account3',
    'Amount': 50
}
bloque2 = Bloque(transaction2, datetime.now().timestamp(), bloque1)
transaction3= {
    'From': 'account3',
    'To': 'account4',
    'Amount': 30
}
bloque3 = Bloque(transaction3, datetime.now().timestamp(), bloque2)

# Create the blockchain by linking the blocks in the correct order
cadena_ejemplo = bloque3

# Verify the validity of the blockchain
validez_cadena = is_valid_blockchain(cadena_ejemplo)
print("Validez de la cadena de bloques:", validez_cadena)

# Verify the validity of a transaction
transaccion = {
    'From': 'account1',
    'To': 'account2',
    'Amount': 5
}
validez_transaccion = is_valid_transaction(transaccion, bloque1)
print("Validez de la transacción:", validez_transaccion)

# Get the account balances
account1_balance = get_account_balance('account1', cadena_ejemplo)
print("Balance de account1:", account1_balance)

account2_balance = get_account_balance('account2', cadena_ejemplo)
print("Balance de account2:", account2_balance)

account3_balance = get_account_balance('account3', cadena_ejemplo)
print("Balance de account3:", account3_balance)

# ***The Merkle Tree***

In [ ]:
class Node:
    def __init__(self, value: str, left_child=None, right_child=None):
        self.value = value
        self.left_child = left_child
        self.right_child = right_child

def arboles_iguales(arbol1,arbol2):
   if (arbol1==None) and (arbol2 == None):
      return True
   else:
      if (arbol1 == None) or (arbol2 ==None) or (arbol1.value != arbol2.value):
         return False
      else:
         return (arboles_iguales(arbol1.left_child,arbol2.left_child) and  arboles_iguales(arbol1.right_child,arbol2.right_child))


def profundidad_arbol(arbol):  # supuesto binario y completo (i.e. todas la ramas maximales son de la misma longitud)
  if (arbol == None):
     return -1
  else:
     return 1 + profundidad_arbol(arbol.left_child)

# El blockchain tienes que hacerlo con la definición última, la buena:
def obtener_hashes(blockchain):
    hashes = [ ]
    bloque = blockchain
    while bloque:
        hashes.append(bloque.cryptographic_hash)
        bloque = bloque.bloque_previo
    return hashes

def build_merkle_tree(blockchain):
    hashes = obtener_hashes(blockchain)
    old_set_of_nodes = fill_set([Node(data) for data in hashes])
    tree_depth = compute_tree_depth(len(old_set_of_nodes))

    for i in range(tree_depth):
        num_nodes = len(old_set_of_nodes)
        new_set_of_nodes = []

        for j in range(0, num_nodes, 2):
            child_node_0 = old_set_of_nodes[j]
            child_node_1 = old_set_of_nodes[j + 1] if j + 1 < num_nodes else child_node_0

            new_node = Node(
                value=calculate_hash(f"{child_node_0.value}{child_node_1.value}"),
                left_child=child_node_0,
                right_child=child_node_1
            )

            new_set_of_nodes.append(new_node)

        old_set_of_nodes = new_set_of_nodes

    return old_set_of_nodes[0]


# def arbol_merkel_valido(arbol_merkel, blockchain):
#    return arboles_iguales(arbol_merkel,build_merckle_tree(blockchain)) # versión ineficaz

# Para la versión más eficaz:

def nodos_por_nivel (arbol_merckle,nivel):
   if nivel > profundidad_arbol(arbol_merckle):
      return [ ]
   else:
      nodos = [ ]
      if (nivel == 0):
         nodos.append(arbol_merckle) # devuelve el nodo raíz
         return nodos
      else:
          nodos_nmenos1 = nodos_por_nivel(arbol_merckle,nivel-1)
          while nodos_nmenos1 != [ ]:
              nodos.append(nodos_nmenos1[0].left_child)
              nodos.append(nodos_nmenos1[0].right_child)
              nodos_nmenos1 = nodos_nmenos1[1:]
          return nodos

# def hashes_por_nivel (arbol_merckle,nivel):
#    return [nodo.value for nodo in nodos_por_nivel(arbol_merckle,nivel)]

def hashes_por_nivel(arbol_merckle, nivel):
    nodes = nodos_por_nivel(arbol_merckle, nivel)
    return [node.value for node in nodes if node is not None]


def arbol_merkle_valido(arbol_merkle,blockchain):
    hashes = obtener_hashes(blockchain)
    old_set_of_nodes = fill_set([Node(data) for data in hashes])
    tree_depth = compute_tree_depth(len(old_set_of_nodes))
    if tree_depth != profundidad_arbol (arbol_merkle):
       return False
    hashes_hojas = hashes_por_nivel(arbol_merkle,tree_depth)
    if hashes != hashes_hojas:
       return False
    for i in range(tree_depth):
        num_nodes = len(old_set_of_nodes)
        new_set_of_nodes = []
        hashes_arbol = hashes_por_nivel(arbol_merkle,tree_depth-i-1)

        for j in range(0, num_nodes, 2):
            child_node_0 = old_set_of_nodes[j]
            child_node_1 = old_set_of_nodes[j + 1] if j + 1 < num_nodes else child_node_0
            new_hash = calculate_hash(f"{child_node_0.value}{child_node_1.value}")
            if new_hash != hashes_arbol[0]:
               return False
            new_node = Node(
                value=new_hash,
                left_child=child_node_0,
                right_child=child_node_1
            )
            new_set_of_nodes.append(new_node)
            hashes_arbol=hashes_arbol[1:]

        old_set_of_nodes = new_set_of_nodes

    return True

def compute_tree_depth(number_of_leaves: int) -> int:
    return math.ceil(math.log2(number_of_leaves))


def is_power_of_2(number_of_leaves: int) -> bool:
    return math.log2(number_of_leaves).is_integer()


def fill_set(list_of_nodes):
    current_number_of_leaves = len(list_of_nodes)
    if is_power_of_2(current_number_of_leaves):
        return list_of_nodes
    total_number_of_leaves = 2 ** compute_tree_depth(current_number_of_leaves)
    if current_number_of_leaves % 2 == 0:
        for i in range(current_number_of_leaves, total_number_of_leaves, 2):
            list_of_nodes = list_of_nodes + [list_of_nodes[-2], list_of_nodes[-1]]
    else:
         for i in range(current_number_of_leaves, total_number_of_leaves):
            list_of_nodes.append(list_of_nodes[-1])
    return list_of_nodes


def calculate_hash(data: str) -> str:
    bytes_data = bytearray(data, "utf-8")
    h = SHA256.new()
    h.update(bytes_data)
    return h.hexdigest()


def print_merkle_tree(node, level=0):
    if node is None:
        return

    print("Level", level, ":")
    print(" " * (level * 4) + "├─", node.value)

    print_merkle_tree(node.left_child, level + 1)
    print_merkle_tree(node.right_child, level + 1)


# Create the initial transaction
initial_transaction = {
    'From': 'genesis',
    'To': 'account1',
    'Amount': 1000
}

# Create the initial block
bloque_inicial = Bloque(initial_transaction, datetime.now().timestamp())

# Create subsequent blocks with transactions and timestamps
transaction1= {
    'From': 'account1',
    'To': 'account2',
    'Amount': 100
}
bloque1 = Bloque(transaction1, datetime.now().timestamp(), bloque_inicial)

transaction2 = {
    'From': 'account2',
    'To': 'account3nezha',
    'Amount': 50
}
bloque2 = Bloque(transaction2, datetime.now().timestamp(), bloque1)
transaction3= {
    'From': 'account3',
    'To': 'account4',
    'Amount': 30
}
bloque3 = Bloque(transaction3, datetime.now().timestamp(), bloque2)

# Create the blockchain by linking the blocks in the correct order
blockchain = bloque3

# Obtain the Merkle tree
merkle_tree = build_merkle_tree(blockchain)

print(profundidad_arbol(merkle_tree)) # Debe escribir 2
print(len(obtener_hashes(blockchain))) # Debe escribir 4
print(compute_tree_depth(len(obtener_hashes(blockchain)))) # Debe escribir 2


# Check if the Merkle tree is valid
merkle_valid = arbol_merkle_valido(merkle_tree, blockchain)
# Print the Merkle tree validity
print("Merkle Tree Validity:", merkle_valid) # Debe escribir True

# Print the Merkle tree
print("\nMerkle Tree:")

print_merkle_tree(merkle_tree)

bad_blockchain = Bloque({'From': 'genesis',
    'To': 'Nezha',
    'Amount': 50000
},111,blockchain.bloque_previo)

merkle_invalid1 = arbol_merkle_valido(merkle_tree, bad_blockchain)
# Print the Merkle tree validity
print("Merkle Tree Validity:", merkle_invalid1) # Debe escribir False

bad_tree = Node('bad_hash',merkle_tree.left_child,merkle_tree.right_child)

merkle_invalid2 = arbol_merkle_valido(bad_tree, blockchain)
# Print the Merkle tree validity
print("Merkle Tree Validity:", merkle_invalid2) # Debe escribir False

merkle_invalid3 = arbol_merkle_valido(bad_tree, bad_blockchain)
# Print the Merkle tree validity
print("Merkle Tree Validity:", merkle_invalid3) # Debe escribir False

# ***signature validation and public_private key***



In [ ]:
class Wallet:
    def __init__(self):
        self.private_key = RSA.generate(2048)
        self.public_key = self.private_key.publickey().export_key()
        self.bitcoin_address = self.generate_bitcoin_address()

    def generate_bitcoin_address(self):
        hash_sha256 = SHA256.new(self.public_key).digest()
        hash_ripemd160 = RIPEMD160.new(hash_sha256).digest()
        bitcoin_address = base58.b58encode(hash_ripemd160)
        return bitcoin_address.decode()

class Transaction:
    def __init__(self, sender_wallet, receiver_bitcoin_address, amount):
        self.sender_wallet = sender_wallet
        self.receiver_bitcoin_address = receiver_bitcoin_address
        self.amount = amount
        self.signature = None

    def sign(self):
        transaction_data = self.generate_data()
        hash_object = SHA256.new(transaction_data)
        signature = pkcs1_15.new(self.sender_wallet.private_key).sign(hash_object)
        self.signature = binascii.hexlify(signature).decode("utf-8")

    def verify_signature(self):
        transaction_data = self.generate_data()
        hash_object = SHA256.new(transaction_data)
        try:
            signature = binascii.unhexlify(self.signature.encode("utf-8"))
            verifier = pkcs1_15.new(self.sender_wallet.private_key)
            verifier.verify(hash_object, signature)
            return True
        except ValueError:
            return False

    def generate_data(self):
        transaction_data = {
            "From": self.sender_wallet.bitcoin_address,
            "To": self.receiver_bitcoin_address,
            "Amount": self.amount
        }
        return convert_transaction_data_to_bytes(transaction_data)

def convert_transaction_data_to_bytes(transaction_data):
    new_transaction_data = transaction_data.copy()
    new_transaction_data["From"] = str(transaction_data["From"])
    new_transaction_data["To"] = str(transaction_data["To"])
    new_transaction_data["Amount"] = str(transaction_data["Amount"])
    return json.dumps(new_transaction_data, indent=2).encode('utf-8')

# Create wallets
nezha_wallet = Wallet()
julio_wallet = Wallet()

transaction = Transaction(nezha_wallet, julio_wallet.bitcoin_address, 100)
transaction.sign()
# Verify the transaction signature
is_signature_valid = transaction.verify_signature()
print("Transaction Signature Valid:", is_signature_valid)  # Should return True

# Modify the transaction amount to make the signature validation fail
transaction.amount = 200  # Change the amount to something different

# Verify the transaction signature
is_signature_valid = transaction.verify_signature()
print("Transaction Signature Valid:", is_signature_valid)  # Should return False

# Print the public and private keys
print("Nezha's Public Key:", nezha_wallet.public_key.decode())
print("Nezha's Private Key:", nezha_wallet.private_key.export_key().decode())
